In [2]:
import pandas as pd
df = pd.read_csv('school_attack1.csv')
df

,school,image,tag_info
0,가거도초등학교,saved_images/image_83196.jpg,"{'total_tag': 4, 'total_label_tag': {'낙엽': 1, ..."
1,가경초등학교,saved_images/image_40152.jpg,"{'total_tag': 0, 'total_label_tag': {}}"
2,가거도초등학교,saved_images/image_83555.jpg,"{'total_tag': 5, 'total_label_tag': {'종이': 2, ..."
3,가거도초등학교,saved_images/image_93295.jpg,"{'total_tag': 1, 'total_label_tag': {'종이': 1}}"


In [3]:
import ast
df['tag_info'] = df['tag_info'].apply(ast.literal_eval)
df

,school,image,tag_info
0,가거도초등학교,saved_images/image_83196.jpg,"{'total_tag': 4, 'total_label_tag': {'낙엽': 1, ..."
1,가경초등학교,saved_images/image_40152.jpg,"{'total_tag': 0, 'total_label_tag': {}}"
2,가거도초등학교,saved_images/image_83555.jpg,"{'total_tag': 5, 'total_label_tag': {'종이': 2, ..."
3,가거도초등학교,saved_images/image_93295.jpg,"{'total_tag': 1, 'total_label_tag': {'종이': 1}}"


In [4]:
df['total_tag'] = df['tag_info'].apply(lambda x: x.get('total_tag', 0))
df['total_tag']

0    4
1    0
2    5
3    1
Name: total_tag, dtype: int64

In [5]:
df['total_label_tag'] = df['tag_info'].apply(lambda x: x.get('total_label_tag', {}))
df['total_label_tag']

0    {'낙엽': 1, '담배꽁초': 2, '종이': 1}
1                               {}
2             {'종이': 2, '담배꽁초': 3}
3                        {'종이': 1}
Name: total_label_tag, dtype: object

In [6]:
# 모든 라벨 추출
all_labels = set()
for label_dict in df['total_label_tag']:
    all_labels.update(label_dict.keys())

# 각 라벨별 컬럼 생성
for label in all_labels:
    df[label] = df['total_label_tag'].apply(lambda x: x.get(label, 0))

df.head()

,school,image,tag_info,total_tag,total_label_tag,낙엽,담배꽁초,종이
0,가거도초등학교,saved_images/image_83196.jpg,"{'total_tag': 4, 'total_label_tag': {'낙엽': 1, ...",4,"{'낙엽': 1, '담배꽁초': 2, '종이': 1}",1,2,1
1,가경초등학교,saved_images/image_40152.jpg,"{'total_tag': 0, 'total_label_tag': {}}",0,{},0,0,0
2,가거도초등학교,saved_images/image_83555.jpg,"{'total_tag': 5, 'total_label_tag': {'종이': 2, ...",5,"{'종이': 2, '담배꽁초': 3}",0,3,2
3,가거도초등학교,saved_images/image_93295.jpg,"{'total_tag': 1, 'total_label_tag': {'종이': 1}}",1,{'종이': 1},0,0,1


In [7]:
df.drop(columns=['tag_info', 'total_label_tag'], inplace=True)
df

,school,image,total_tag,낙엽,담배꽁초,종이
0,가거도초등학교,saved_images/image_83196.jpg,4,1,2,1
1,가경초등학교,saved_images/image_40152.jpg,0,0,0,0
2,가거도초등학교,saved_images/image_83555.jpg,5,0,3,2
3,가거도초등학교,saved_images/image_93295.jpg,1,0,0,1


In [8]:
drain_counts = df.groupby('school')['image'].count().reset_index(name='살린 배수구 개수')
drain_counts

,school,살린 배수구 개수
0,가거도초등학교,3
1,가경초등학교,1


In [9]:
price = 10000
drain_counts['price'] = drain_counts['살린 배수구 개수'] * price
drain_counts

,school,살린 배수구 개수,price
0,가거도초등학교,3,30000
1,가경초등학교,1,10000


In [10]:
item_prices = {
    '요아정': 4500,
    '마라탕':13000,
    '아이스크림':1500
}

for items, price in item_prices.items():
    drain_counts[items] = (drain_counts['price']/price).astype(int)

drain_counts

,school,살린 배수구 개수,price,요아정,마라탕,아이스크림
0,가거도초등학교,3,30000,6,2,20
1,가경초등학교,1,10000,2,0,6


In [11]:
drain_counts['총 살린 배수구 개수'] = drain_counts['살린 배수구 개수'].count()
drain_counts

,school,살린 배수구 개수,price,요아정,마라탕,아이스크림,총 살린 배수구 개수
0,가거도초등학교,3,30000,6,2,20,2
1,가경초등학교,1,10000,2,0,6,2


In [12]:
drain_counts = drain_counts.drop(columns='총 살린 배수구 개수')
drain_counts

,school,살린 배수구 개수,price,요아정,마라탕,아이스크림
0,가거도초등학교,3,30000,6,2,20
1,가경초등학교,1,10000,2,0,6


In [13]:
total = pd.DataFrame({
    'school':['총 합'],
    '살린 배수구 개수': [drain_counts['살린 배수구 개수'].sum()],
    'price': [drain_counts['price'].sum()]
})

In [15]:
for item, price in item_prices.items():
    total[item] = [int(total['price'][0]/price)]

print(total)

  school  살린 배수구 개수  price  요아정  마라탕  아이스크림
0    총 합          4  40000    8    3     26


In [16]:
total

,school,살린 배수구 개수,price,요아정,마라탕,아이스크림
0,총 합,4,40000,8,3,26
